# Import Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [2]:
# Set image parameters
IMAGE_SIZE = 150
BATCH_SIZE = 16


In [ ]:
# Set data directories
training_data_dir = r"C:\Users\Microsoft\Desktop\DL project\data of mountains, buildins etc\seg_train"
testing_data_dir = r"C:\Users\Microsoft\Desktop\DL project\data of mountains, buildins etc\seg_test"
validation_data_dir = r"C:\Users\Microsoft\Desktop\DL project\data of mountains, buildins etc\seg_pred"

In [ ]:
# Create data generators
training_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    training_data_dir,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

testing_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    testing_data_dir,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    validation_data_dir,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

In [ ]:
class_names = training_dataset.class_names
print(class_names)

In [6]:
# Data augmentation and preprocessing layers
data_augmentation = tf.keras.Sequential([
    layers.Rescaling(1.0 / 255),
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

In [7]:
# Apply data augmentation and resizing
train_data = training_dataset.map(lambda x, y: (data_augmentation(x), y))
valid_data = validation_dataset.map(lambda x, y: (data_augmentation(x), y))
test_data = testing_dataset.map(lambda x, y: (data_augmentation(x), y))


In [8]:
# Pre-trained MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')

C:\Users\Microsoft\AppData\Local\Temp\ipykernel_13348\2572960981.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 15s 2us/step


In [9]:
# Freeze the base model
base_model.trainable = False


In [10]:
# Add classification head
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(class_names), activation='softmax')

In [11]:
# Build the model
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

In [12]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
# Train the model
epochs = 10
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=valid_data
)

Epoch 1/10


C:\Users\Microsoft\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:827: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


878/878 ━━━━━━━━━━━━━━━━━━━━ 234s 263ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/10
878/878 ━━━━━━━━━━━━━━━━━━━━ 2192s 2s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10
878/878 ━━━━━━━━━━━━━━━━━━━━ 255s 291ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/10
878/878 ━━━━━━━━━━━━━━━━━━━━ 257s 293ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
878/878 ━━━━━━━━━━━━━━━━━━━━ 258s 294ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/10
878/878 ━━━━━━━━━━━━━━━━━━━━ 250s 284ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
878/878 ━━━━━━━━━━━━━━━━━━━━ 245s 279ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data)
print("Test accuracy:", test_accuracy)
print("Test loss:", test_loss)

In [ ]:
model.save("Semester_end.keras")